In [2]:
import numpy as np
import pandas as pd

In [3]:
def year_apply(year: int) -> int:
    if 0 <= year < 16:
        return year + 2000
    elif 52 <= year <= 99:
        return year + 1900
    else:
        return year

In [ ]:
def data_pipe(dvf_csv):
    train_df = pd.read_csv(dvf_csv)
    train_df.fillna('undefined', inplace=True)
    train_df['registration_year'] = train_df['registration_year'].apply(lambda x: year_apply(x))
    from sklearn.preprocessing import LabelEncoder

    le = LabelEncoder()
    for column in ['model', 'type', 'gearbox', 'fuel', 'brand']:
        train_df[column] = le.fit_transform(train_df[column])
train_df

In [60]:
train_df = pd.read_csv('train.csv')
train_df.fillna('undefined', inplace=True)
train_df['registration_year'] = train_df['registration_year'].apply(lambda x: year_apply(x))

In [61]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for column in ['model', 'type', 'gearbox', 'fuel', 'brand']:
    train_df[column] = le.fit_transform(train_df[column])
train_df

,Unnamed: 0,engine_capacity,type,registration_year,gearbox,power,model,mileage,fuel,brand,damage,zipcode,insurance_price,price
0,48298,2.0,0,2006,0,140,58,150000,2,5,0.0,49191,380.0,4267
1,81047,undefined,7,2016,2,0,235,150000,5,20,undefined,45896,undefined,2457
2,92754,2.2,3,2010,1,175,154,125000,1,10,0.0,59229,930.0,10374
3,46007,undefined,7,2000,0,265,40,150000,2,10,0.0,39365,680.0,7098
4,76981,undefined,1,2003,1,109,8,150000,2,25,0.0,55271,undefined,2365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,50429,1.4,3,2006,1,75,117,90000,2,38,0.0,35745,500.0,4686
49996,64425,1.3,5,2004,1,60,103,150000,2,10,0.0,60386,undefined,864
49997,90761,undefined,3,1996,1,150,15,150000,2,2,0.0,28309,130.0,2275
49998,39709,undefined,3,2007,1,122,6,100000,1,2,0.0,83623,500.0,8144


In [62]:
x_idx = train_df['Unnamed: 0']
X = train_df.drop(['zipcode', 'price', 'Unnamed: 0'], axis=1)
y = train_df['price']

In [63]:
X.replace('undefined', -1, inplace=True)

In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    train_size=0.7,
                                                    test_size = 0.3, random_state=100)

In [66]:
def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [73]:
from sklearn.ensemble import BaggingRegressor
from sklearn.linear_model import RidgeClassifier

lm = BaggingRegressor(n_estimators=30, n_jobs=4, random_state=1)
lm.fit(X_train,y_train)

BaggingRegressor(n_estimators=30, n_jobs=4, random_state=1)

In [74]:
y_pred_test=lm.predict(X_test)
y_pred_train=lm.predict(X_train)

In [75]:
mean_absolute_percentage_error(y_test, y_pred_test)

32.10632318483748